In [1]:
CHROMA_PATH = "msd_chroma"

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [3]:
def get_embeddings():
   model_kwargs = {'device': 'cuda'}
   embeddings_hf = HuggingFaceEmbeddings(
       model_name='intfloat/multilingual-e5-large',
       model_kwargs=model_kwargs
   )
   return embeddings_hf

In [25]:
PROMPT_TEMPLATE = """
Вы — медицинский ассистент для врачей-специалистов. Вы предоставляете информацию исключительно на основе статей из MSD справочника и клинических рекомендаций по лечению определенных заболеваний. Ваши ответы должны быть точными, основанными на этих источниках, и представлены в понятной и сжатой форме. Если информация отсутствует в указанных источниках, сообщите об этом и избегайте предположений.

Ответь на вопрос базируясь только на этом контексте: 

{context}

---

Вопрос: {question}
"""

In [26]:
import argparse
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_community.vectorstores import Chroma


In [27]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embeddings())


In [37]:
query_text='Какие симптомы характерны для тендинопатии вращательной манжеты плеча?'
results = db.similarity_search_with_relevance_scores(query_text, k=5)
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Нет фрагментов текста, на которые можно опираться для ответа.")

In [38]:
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)
print(f"Полученный промпт {prompt}")

Полученный промпт Human: 
Вы — медицинский ассистент для врачей-специалистов. Вы предоставляете информацию исключительно на основе статей из MSD справочника и клинических рекомендаций по лечению определенных заболеваний. Ваши ответы должны быть точными, основанными на этих источниках, и представлены в понятной и сжатой форме. Если информация отсутствует в указанных источниках, сообщите об этом и избегайте предположений.

Ответь на вопрос базируясь только на этом контексте: 

конечностей с утолщением кожи пальцев. Отмечается также выраженная полиартралгия. Иногда первыми проявлениями заболевания могут быть нарушения со
стороны желудочно-кишечного тракта (например, изжога, дисфагия) или со стороны дыхательной системы (например, диспноэ).
Поражение кожи и ногтей
Отек кожи обычно симметричный, прогрессирует до индурации. Он может проявляться только на пальцах (склеродактилия) и кистях, но может поражать обширные
участки или все тело. Кожа становится туго натянутой, напряженной, блестящей и

In [39]:
model = ChatOpenAI(temperature=0.7, base_url="http://localhost:1234/v1", api_key="not-needed")
response_text = model.predict(prompt)


sources = [doc.metadata.get("source", None) for doc, _score in results]
formatted_response = f"Ответ: {response_text}\nДанные взяты из: {sources}"
print(formatted_response)

Ответ: Тендинопатия вращательной манжеты плеча - это распространенное заболевание, которое вызывает боль и ограничение движений в области плеча. Характерные симптомы тендинопатии вращательной манжеты плеча включают:

* Боль в области плеча или верхней части руки, особенно при движении руки вверх или вниз.
* Ограничение движений в области плеча, что может проявляться как затруднение подъема руки или ее поворота.
* Затвердение и отек в области вращательной манжеты плеча, которое можно заметить при пальпации.
* Боль при сгибании или разгибании руки.

Некоторые люди также могут испытывать чувство скованности или тяжести в верхней части руки или плеча после отдыха. Симптомы тендинопатии вращательной манжеты плеча можно рассматривать как вариант диапазона проявлений, которые не являются исключительно характерными для этого заболевания.

Тендинопатия вращательной манжеты плеча вызвана повреждением или воспалением тендонов, которые соединяют мышцы с кости. Это может быть спровоцировано повторя